In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [48]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [49]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [50]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [51]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [52]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [53]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [54]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [55]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [56]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [57]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\Al\RecSystems\Lesson5\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [58]:

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [59]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [60]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [61]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
user_feat.shape

(2497, 7)

In [63]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [64]:
item_feat.shape

(5001, 6)

## Encoding features

In [65]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [66]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

rez_list_bpr = list(zip( params_list, bpr_list))
print(rez_list_bpr)

In [92]:
import itertools
no_components_list = [40, 50, 60, 70]
loss_list = ["bpr", "warp"]
item_alpha_list  = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
user_alpha_list  = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

params_list = list(itertools.product(no_components_list, loss_list, item_alpha_list, user_alpha_list))
#params_list = list(itertools.product(no_components_list, item_alpha_list, user_alpha_list))
print(params_list)

[(40, 'bpr', 0.1, 0.1), (40, 'bpr', 0.1, 0.2), (40, 'bpr', 0.1, 0.3), (40, 'bpr', 0.1, 0.4), (40, 'bpr', 0.1, 0.5), (40, 'bpr', 0.1, 0.6), (40, 'bpr', 0.2, 0.1), (40, 'bpr', 0.2, 0.2), (40, 'bpr', 0.2, 0.3), (40, 'bpr', 0.2, 0.4), (40, 'bpr', 0.2, 0.5), (40, 'bpr', 0.2, 0.6), (40, 'bpr', 0.3, 0.1), (40, 'bpr', 0.3, 0.2), (40, 'bpr', 0.3, 0.3), (40, 'bpr', 0.3, 0.4), (40, 'bpr', 0.3, 0.5), (40, 'bpr', 0.3, 0.6), (40, 'bpr', 0.4, 0.1), (40, 'bpr', 0.4, 0.2), (40, 'bpr', 0.4, 0.3), (40, 'bpr', 0.4, 0.4), (40, 'bpr', 0.4, 0.5), (40, 'bpr', 0.4, 0.6), (40, 'bpr', 0.5, 0.1), (40, 'bpr', 0.5, 0.2), (40, 'bpr', 0.5, 0.3), (40, 'bpr', 0.5, 0.4), (40, 'bpr', 0.5, 0.5), (40, 'bpr', 0.5, 0.6), (40, 'bpr', 0.6, 0.1), (40, 'bpr', 0.6, 0.2), (40, 'bpr', 0.6, 0.3), (40, 'bpr', 0.6, 0.4), (40, 'bpr', 0.6, 0.5), (40, 'bpr', 0.6, 0.6), (40, 'warp', 0.1, 0.1), (40, 'warp', 0.1, 0.2), (40, 'warp', 0.1, 0.3), (40, 'warp', 0.1, 0.4), (40, 'warp', 0.1, 0.5), (40, 'warp', 0.1, 0.6), (40, 'warp', 0.2, 0.1), (40

In [93]:
pres_list = []

In [94]:
%%time

for i in range(len(params_list)+1):
    model = LightFM(no_components=params_list[i][0],
            loss=params_list[i][1],#'warp', # "logistic","bpr", "warp"
            learning_rate=0.01, 
            item_alpha=params_list[i][2],
            user_alpha=params_list[i][3], 
            random_state=42,
            k=5,
            n=15,
            max_sampled=100)

    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 
    
    
    user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())
    
    item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())
    
    
    # подготавливаемм id для юзеров и товаров в порядке пар user-item
    users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
    items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)
    
    
    # модель возвращает меру/скор похожести между соответствующим пользователем и товаром
    predictions = model.predict(user_ids=users_ids_row,
                    item_ids=items_ids_row,
                    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                    num_threads=10)
    
    # добавляем наш полученный скор в трейн датафрейм
    data_train_filtered['score'] = predictions
    
    # создаем предикт датафрейм в формате списка това
    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()
    
    # объединяем предикт и тест датасет для подсчета precision
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    
    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
    #logistic_list.append(precision)
    pres_list.append(precision)
    #print(f"Precision: {precision}")
    

Epoch: 100%|██████████| 20/20 [04:53<00:00, 14.67s/it]


Epoch: 100%|██████████| 20/20 [06:33<00:00, 19.68s/it]


IndexError: list index out of range

In [95]:
pres_list

[0.14436781609195218,
 0.148111658456484,
 0.14811165845648402,
 0.14939244663382378,
 0.14880131362889776,
 0.1396387520525434,
 0.1420032840722478,
 0.1374712643678144,
 0.13520525451559778,
 0.1318555008210166,
 0.13047619047618908,
 0.1313628899835782,
 0.14101806239737102,
 0.13175697865352892,
 0.11106732348111571,
 0.06830870279146177,
 0.06939244663382628,
 0.06929392446633859,
 0.13963875205254345,
 0.13087027914613975,
 0.05934318555008257,
 0.061412151067323964,
 0.059934318555008664,
 0.05993431855500868,
 0.13589490968801146,
 0.130771756978652,
 0.06200328407225009,
 0.057175697865353474,
 0.057077175697865794,
 0.057274220032841175,
 0.12220032840722381,
 0.13008210180623828,
 0.055008210180624385,
 0.054909688013136705,
 0.05540229885057513,
 0.05490968801313672,
 0.07471264367816102,
 0.07707717569786536,
 0.08101806239737269,
 0.08712643678160892,
 0.10072249589490898,
 0.12781609195402163,
 0.11668308702791362,
 0.07323481116584578,
 0.07215106732348132,
 0.072840722

In [119]:
columns_params = ['no_components', 'loss', 'item_alpha', 'user_alpha']
#index = [1, 2, 3, 4]
df_params = pd.DataFrame(params_list, columns = columns_params)
df_params

,no_components,loss,item_alpha,user_alpha
0,40,bpr,0.1,0.1
1,40,bpr,0.1,0.2
2,40,bpr,0.1,0.3
3,40,bpr,0.1,0.4
4,40,bpr,0.1,0.5
...,...,...,...,...
283,70,warp,0.6,0.2
284,70,warp,0.6,0.3
285,70,warp,0.6,0.4
286,70,warp,0.6,0.5


In [120]:
columns_pres = ['presision']
df_pres = pd.DataFrame(pres_list, columns = columns_pres)
df_pres                  

,presision
0,0.144368
1,0.148112
2,0.148112
3,0.149392
4,0.148801
...,...
283,0.127422
284,0.082890
285,0.074811
286,0.072644


In [129]:
all_columns = ['no_components', 'loss', 'item_alpha', 'user_alpha', 'presision']
df_rezult = pd.concat([df_params, df_pres], ignore_index=True, axis = 1)
df_rezult.columns = all_columns
df_rezult.sort_values('presision', ascending=False, inplace=True)

df_rezult

,no_components,loss,item_alpha,user_alpha,presision
147,60,bpr,0.1,0.4,0.151067
148,60,bpr,0.1,0.5,0.150378
3,40,bpr,0.1,0.4,0.149392
149,60,bpr,0.1,0.6,0.148900
4,40,bpr,0.1,0.5,0.148801
...,...,...,...,...,...
249,70,bpr,0.6,0.4,0.054516
251,70,bpr,0.6,0.6,0.054516
177,60,bpr,0.6,0.4,0.054417
105,50,bpr,0.6,0.4,0.053924


# Getting embeddings

## вектора по пользователям

In [47]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [48]:
user_emb[0].shape # biases

(2497,)

In [49]:
user_emb[1].shape # users vectors

(2497, 40)

## вектора по товарам

In [50]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [51]:
item_emb[0].shape # biases

(5001,)

In [52]:
item_emb[1].shape # items vectors

(5001, 40)

# Evaluation -> Train precision

In [4]:
# мы можем использовать встроенные метрики lightFM

train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

NameError: name 'precision_at_k' is not defined

# Predict

In [54]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [55]:
users_ids_row[:10]

array([2371, 1363, 1363, 1363, 1363, 1171, 1171, 1171, 1171, 1171])

In [56]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [57]:
# добавляем наш полученный скор в трейн датафрейм
data_train_filtered['score'] = predictions

In [58]:
data_train_filtered.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,score
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99,-0.023003
11,1364,26984896261,1,999999,1,2.19,31742,0.00,1520,1,0.0,0.0,2.19,-0.983616
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0,2.99,-0.983616
13,1364,26984896261,1,999999,1,3.09,31742,0.00,1520,1,0.0,0.0,3.09,-0.983616
14,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50,-1.003003


In [59]:
# создаем предикт датафрейм в формате списка това
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [60]:
predict_result.head()

,user_id,item_id
0,1,"[1029743, 6034857, 952163, 1004906, 1088462, 8..."
1,2,"[1106523, 1075368, 6919458, 952163, 899624, 83..."
2,3,"[1106523, 983584, 5585510, 899624, 946839, 839..."
3,4,"[1029743, 1075368, 1052294, 7431408, 970760, 1..."
4,5,"[1126899, 1029743, 6034991, 825659, 1112387, 1..."


In [61]:
# объединяем предикт и тест датасет для подсчета precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [62]:
df_result_for_metrics.head()

,user_id,actual,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1029743, 6034857, 952163, 1004906, 1088462, 8..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 983584, 5585510, 899624, 946839, 839..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1070820, 1029743, 1126899, 1121393, 9524291, ..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 1126899, 1106523, 1072494, 9835880, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1070820, 1029743, 5585510, 1139835, ..."


### Test with custom precision func

In [44]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.14357963875205076


# Links

Neural networks for RS: http://d2l.ai/chapter_recommender-systems/mf.html

LigthFM -> https://arxiv.org/pdf/1507.08439.pdf

https://making.lyst.com/lightfm/docs/home.html


# Домашнее задание

#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией
    Подбор параметров:
    - Loss
    - no_components
    - regularization